In [1]:
# Importando as bibliotecas necessárias
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Carregando e preparando os dados CIFAR-10
(x_treino, y_treino), (x_teste, y_teste) = cifar10.load_data()
x_treino = x_treino.astype('float32') / 255.0
x_teste = x_teste.astype('float32') / 255.0
y_treino = to_categorical(y_treino)
y_teste = to_categorical(y_teste)

In [3]:
# Definição do Gerador
def criar_gerador():
    modelo = models.Sequential([
        layers.Dense(256, activation='relu', input_dim=100),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(1024, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(32 * 32 * 3, activation='tanh'),
        layers.Reshape((32, 32, 3))
    ])
    return modelo

In [4]:
# Definição do Discriminador
def criar_discriminador():
    modelo = models.Sequential([
        layers.Flatten(input_shape=(32, 32, 3)),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    modelo.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])
    return modelo

In [5]:
gerador = criar_gerador()
discriminador = criar_discriminador()

In [6]:
# Configurando e compilando a GAN
entrada_gan = Input(shape=(100,))
imagem_falsa = gerador(entrada_gan)
discriminador.trainable = False
decisao_falsa = discriminador(imagem_falsa)
gan = models.Model(entrada_gan, decisao_falsa)
gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

In [7]:
# Função para treinar a GAN
def treinar_gan(gerador, discriminador, gan, epochs, batch_size):
    for epoch in range(epochs):
        # Amostras de ruído e imagens geradas
        ruído = np.random.normal(0, 1, (batch_size, 100))
        imagens_geradas = gerador.predict(ruído)

        # Amostras de imagens reais
        indices = np.random.randint(0, x_treino.shape[0], batch_size)
        imagens_reais = x_treino[indices]

        # Treinar o discriminador
        perda_real = discriminador.train_on_batch(imagens_reais, np.ones((batch_size, 1)))
        perda_falsa = discriminador.train_on_batch(imagens_geradas, np.zeros((batch_size, 1)))
        perda_discriminador = 0.5 * np.add(perda_real, perda_falsa)

        # Treinar o gerador
        ruído = np.random.normal(0, 1, (batch_size, 100))
        perda_gerador = gan.train_on_batch(ruído, np.ones((batch_size, 1)))

        print(f'Epoch {epoch + 1}/{epochs}, Perda Discriminador: {perda_discriminador}, Perda Gerador: {perda_gerador}')

In [8]:
# Treinando a GAN
treinar_gan(gerador, discriminador, gan, epochs=100, batch_size=128)

4/4 [==============================] - 1s 8ms/step
Epoch 1/100, Perda Discriminador: [0.53713927 0.625     ], Perda Gerador: 0.6687787771224976
4/4 [==============================] - 0s 9ms/step
Epoch 2/100, Perda Discriminador: [0.28668665 1.        ], Perda Gerador: 0.6915053725242615
4/4 [==============================] - 0s 10ms/step
Epoch 3/100, Perda Discriminador: [0.21650529 1.        ], Perda Gerador: 0.7703138589859009
4/4 [==============================] - 0s 10ms/step
Epoch 4/100, Perda Discriminador: [0.16351692 1.        ], Perda Gerador: 0.7787222862243652
4/4 [==============================] - 0s 10ms/step
Epoch 5/100, Perda Discriminador: [0.12678736 1.        ], Perda Gerador: 0.8178359270095825
4/4 [==============================] - 0s 9ms/step
Epoch 6/100, Perda Discriminador: [0.10520284 1.        ], Perda Gerador: 0.8202871084213257
4/4 [==============================] - 0s 10ms/step
Epoch 7/100, Perda Discriminador: [0.09255695 1.        ], Perda Gerador: 0.84998

In [9]:
# Gerar novas imagens
imagens_geradas = gerador.predict(np.random.normal(0, 1, (10000, 100)))
x_treino_augmented = np.concatenate([x_treino, imagens_geradas])
y_treino_augmented = np.concatenate([y_treino, y_treino[:10000]])  

313/313 [==============================] - 3s 9ms/step


In [10]:
# Definição do modelo CNN
def criar_modelo_cnn():
    modelo = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return modelo

In [11]:
# Definindo a função para criar o modelo LSTM
def criar_modelo_lstm(taxa_aprendizado=0.001):
    modelo = models.Sequential([
        layers.Bidirectional(layers.LSTM(64, return_sequences=True, dropout=0.25), input_shape=(32, 96)),
        layers.Bidirectional(layers.LSTM(64, dropout=0.25)),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    otimizador = Adam(learning_rate=taxa_aprendizado)
    modelo.compile(optimizer=otimizador, loss='categorical_crossentropy', metrics=['accuracy'])
    return modelo

In [12]:
# Data Augmentation para o modelo CNN
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [13]:
# Early Stopping para o modelo LSTM
early_stopping_lstm = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [15]:
# Treinamento do modelo CNN
modelo_cnn = criar_modelo_cnn()
historico_cnn = modelo_cnn.fit(datagen.flow(x_treino, y_treino, batch_size=32),
                               steps_per_epoch=len(x_treino) / 32,
                               epochs=10,
                               validation_data=(x_teste, y_teste))

Epoch 1/10
1562/1562 [==============================] - 42s 26ms/step - loss: 1.7933 - accuracy: 0.3361 - val_loss: 1.4007 - val_accuracy: 0.4924
Epoch 2/10
1562/1562 [==============================] - 41s 26ms/step - loss: 1.5190 - accuracy: 0.4457 - val_loss: 1.2642 - val_accuracy: 0.5363
Epoch 3/10
1562/1562 [==============================] - 41s 26ms/step - loss: 1.4017 - accuracy: 0.4948 - val_loss: 1.2057 - val_accuracy: 0.5692
Epoch 4/10
1562/1562 [==============================] - 41s 26ms/step - loss: 1.3360 - accuracy: 0.5200 - val_loss: 1.1513 - val_accuracy: 0.5893
Epoch 5/10
1562/1562 [==============================] - 40s 26ms/step - loss: 1.2802 - accuracy: 0.5427 - val_loss: 1.1635 - val_accuracy: 0.5895
Epoch 6/10
1562/1562 [==============================] - 41s 26ms/step - loss: 1.2419 - accuracy: 0.5565 - val_loss: 1.0218 - val_accuracy: 0.6375
Epoch 7/10
1562/1562 [==============================] - 41s 26ms/step - loss: 1.2008 - accuracy: 0.5724 - val_loss: 1.0213 -

In [16]:
# Reformatação dos dados para o modelo LSTM
x_treino_lstm = x_treino.reshape(x_treino.shape[0], 32, 32 * 3)
x_teste_lstm = x_teste.reshape(x_teste.shape[0], 32, 32 * 3)

In [17]:
# Treinamento do modelo LSTM
modelo_lstm = criar_modelo_lstm()
historico_lstm = modelo_lstm.fit(x_treino_lstm, y_treino, epochs=10, validation_data=(x_teste_lstm, y_teste), callbacks=[early_stopping_lstm])

Epoch 1/10
1563/1563 [==============================] - 89s 53ms/step - loss: 1.8677 - accuracy: 0.3173 - val_loss: 1.5881 - val_accuracy: 0.4276
Epoch 2/10
1563/1563 [==============================] - 67s 43ms/step - loss: 1.6383 - accuracy: 0.4132 - val_loss: 1.4915 - val_accuracy: 0.4547
Epoch 3/10
1563/1563 [==============================] - 79s 50ms/step - loss: 1.5393 - accuracy: 0.4474 - val_loss: 1.3960 - val_accuracy: 0.4922
Epoch 4/10
1563/1563 [==============================] - 61s 39ms/step - loss: 1.4716 - accuracy: 0.4762 - val_loss: 1.3534 - val_accuracy: 0.5126
Epoch 5/10
1563/1563 [==============================] - 62s 40ms/step - loss: 1.4267 - accuracy: 0.4940 - val_loss: 1.3013 - val_accuracy: 0.5320
Epoch 6/10
1563/1563 [==============================] - 62s 40ms/step - loss: 1.3794 - accuracy: 0.5095 - val_loss: 1.2897 - val_accuracy: 0.5384
Epoch 7/10
1563/1563 [==============================] - 61s 39ms/step - loss: 1.3402 - accuracy: 0.5252 - val_loss: 1.2742 -

In [18]:
# Avaliação dos modelos
perda_cnn, acuracia_cnn = modelo_cnn.evaluate(x_teste, y_teste)
perda_lstm, acuracia_lstm = modelo_lstm.evaluate(x_teste_lstm, y_teste)

313/313 [==============================] - 4s 12ms/step - loss: 1.2268 - accuracy: 0.5668


In [19]:
print(f"Acurácia do modelo CNN no teste: {acuracia_cnn:.2f}")
print(f"Acurácia do modelo (RNN com LSTM) no teste: {acuracia_lstm:.2f}")

Acurácia do modelo CNN no teste: 0.65
Acurácia do modelo (RNN com LSTM) no teste: 0.57
